# Front Product Analytics Exercise

In [182]:
import pandas as pd
import plotly as py
import plotly.express as px

### Data Prep

In [183]:
# Read the data
df_raw = pd.read_csv("./data/nps_survey.csv")
print(df_raw.shape)
df_raw.head()

(20268, 7)


,is_admin,id,score,created_at,surveyed_at,sessions,messages
0,1,1474,9,2017-01-01T00:46:09Z,2017-02-01T00:46:09Z,4,8
1,1,1129,10,2017-01-01T05:04:09Z,2017-02-01T05:04:09Z,3,8
2,0,3337,10,2017-01-01T07:20:06Z,2017-02-01T07:20:06Z,9,42
3,1,1873,1,2017-01-01T07:31:40Z,2017-02-01T07:31:40Z,35,135
4,0,2720,9,2017-01-01T12:10:27Z,2017-02-01T12:10:27Z,4,13


In [211]:
# Create a user_segment column with All Users, Admin, and Regular
df_raw['user_segment'] = df_raw['is_admin'].apply(lambda x: 'Admin' if x == 1 else 'Regular')
df_raw['creation_month_cohort'] = pd.to_datetime(df_raw['created_at']).dt.to_period('M')
df_all_users = df_raw.copy()
df_all_users['user_segment'] = 'All Users'
df = pd.concat([df_raw, df_all_users])

# Create a response_segment column with Promoter, Detractor, and Neither
df['response_segment'] = df['score'].apply(lambda x: 'Promoter' if x >= 9 else ('Detractor' if x <= 6 else 'Neither'))

print(df.shape)
print(df['user_segment'].value_counts())
df.head()

(40536, 10)
user_segment
All Users    20268
Regular      13112
Admin         7156
Name: count, dtype: int64


/var/folders/l7/904rpb4x1qq_p0g62p7ss_yr0000gn/T/ipykernel_47009/2680309642.py:3: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,is_admin,id,score,created_at,surveyed_at,sessions,messages,user_segment,creation_month_cohort,response_segment
0,1,1474,9,2017-01-01T00:46:09Z,2017-02-01T00:46:09Z,4,8,Admin,2017-01,Promoter
1,1,1129,10,2017-01-01T05:04:09Z,2017-02-01T05:04:09Z,3,8,Admin,2017-01,Promoter
2,0,3337,10,2017-01-01T07:20:06Z,2017-02-01T07:20:06Z,9,42,Regular,2017-01,Promoter
3,1,1873,1,2017-01-01T07:31:40Z,2017-02-01T07:31:40Z,35,135,Admin,2017-01,Detractor
4,0,2720,9,2017-01-01T12:10:27Z,2017-02-01T12:10:27Z,4,13,Regular,2017-01,Promoter


### 1. Compute the NPS of all the monthly cohorts, grouped by date of creation of the user.


In [186]:
# Calculate monthly NPS for all users

df_monthly_nps = (
    (df.groupby(['user_segment', 'creation_month_cohort'])['response_segment'].apply(lambda x: (x == 'Promoter').sum() - (x == 'Detractor').sum()) 
     / df.groupby(['user_segment', 'creation_month_cohort'])['id'].count())*100
).reset_index(name='nps')

df_monthly_nps.sort_values(['user_segment', 'creation_month_cohort'], inplace=True)

df_monthly_nps[df_monthly_nps['user_segment'] == 'All Users']

/var/folders/l7/904rpb4x1qq_p0g62p7ss_yr0000gn/T/ipykernel_47009/849192182.py:3: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,user_segment,creation_month_cohort,nps
19,All Users,2017-01,13.000000
20,All Users,2017-02,14.018692
21,All Users,2017-03,12.953368
22,All Users,2017-04,11.923688
23,All Users,2017-05,12.992701
24,All Users,2017-06,11.966988
25,All Users,2017-07,11.994949
26,All Users,2017-08,10.000000
27,All Users,2017-09,11.038251
28,All Users,2017-10,10.975610


### 2. Display the trend of NPS over time by month for the 3 following segments: all users; admins only; regular users only.


In [187]:
# Line Chart of NPS by User Segment
df_monthly_nps['creation_month_cohort'] = df_monthly_nps['creation_month_cohort'].astype(str)

fig = px.line(df_monthly_nps, x="creation_month_cohort", y="nps", 
                  color="user_segment", 
                  line_group="user_segment",
                  title='NPS by User Segment'
                )
fig.show()

In [188]:
# % Change from Jan 2017 to July 2018
df_monthly_nps_1_17 = df_monthly_nps[df_monthly_nps['creation_month_cohort'] == '2017-01']
df_monthly_nps_7_18 = df_monthly_nps[df_monthly_nps['creation_month_cohort'] == '2018-07']

df_change_nps = df_monthly_nps_1_17.merge(df_monthly_nps_7_18, on='user_segment', suffixes=('_1_17', '_7_18'))

# Calculate the change in NPS
df_change_nps['nps_change'] = (df_change_nps['nps_7_18'] - df_change_nps['nps_1_17'])/ abs(df_change_nps['nps_1_17'])

df_change_nps


,user_segment,creation_month_cohort_1_17,nps_1_17,creation_month_cohort_7_18,nps_7_18,nps_change
0,Admin,2017-01,34.8,2018-07,42.045455,0.208203
1,All Users,2017-01,13.0,2018-07,10.000000,-0.230769
2,Regular,2017-01,-8.8,2018-07,-2.155172,0.755094


### 3. How can you explain the paradoxical trends in Front’s NPS surfaced in the above question?

In [220]:
# % Breakdown of Survey Responses by User Segment - Data Prep

df_monthly_responses_usegments = (df_raw.groupby(['user_segment', 'creation_month_cohort'])['id'].count()
                                .reset_index()
                                .rename(columns={'id': 'response_count'}))
df_monthly_responses_usegments['pct_of_responses'] = df_monthly_responses_usegments.groupby('creation_month_cohort')['response_count'].transform(lambda x: x / x.sum())
df_monthly_responses_usegments.sort_values(['creation_month_cohort', 'user_segment']).head()

,user_segment,creation_month_cohort,response_count,pct_of_responses
0,Admin,2017-01,250,0.500000
19,Regular,2017-01,250,0.500000
1,Admin,2017-02,263,0.491589
20,Regular,2017-02,272,0.508411
2,Admin,2017-03,275,0.474957


In [221]:
# % Breakdown of Survey Responses by User Segment - Chart

stage_table = df_monthly_responses_usegments
stage_table['creation_month_cohort'] = stage_table['creation_month_cohort'].astype(str)

fig = px.bar(stage_table, x="creation_month_cohort", y="pct_of_responses",
                color="user_segment",
                text_auto='.0%',
                title='% of Survey Responses by User Segment',
                color_discrete_map={
                    'Admin': '#646efa',
                    'Regular': '#00cc96'
                }
                )
fig.show()
